In [12]:
# Load data from chapter2
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [13]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [14]:
loss_fn = nn.CrossEntropyLoss()

In [15]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [26]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) 
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [27]:
def test_loop(dataloader, model, loss_fn, device):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [29]:
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    test_loop(test_dataloader, model, loss_fn, device)
print("Done!")

Epoch 1
-------------------------------
loss: 1.860264  [   64/60000]
loss: 1.825199  [ 6464/60000]
loss: 1.695397  [12864/60000]
loss: 1.755359  [19264/60000]
loss: 1.644528  [25664/60000]
loss: 1.598019  [32064/60000]
loss: 1.621766  [38464/60000]
loss: 1.524733  [44864/60000]
loss: 1.555834  [51264/60000]
loss: 1.446906  [57664/60000]
Test Error: 
 Accuracy: 62.3%, Avg loss: 1.469162 

Epoch 2
-------------------------------
loss: 1.537034  [   64/60000]
loss: 1.505114  [ 6464/60000]
loss: 1.349840  [12864/60000]
loss: 1.437512  [19264/60000]
loss: 1.316985  [25664/60000]
loss: 1.315450  [32064/60000]
loss: 1.328013  [38464/60000]
loss: 1.258779  [44864/60000]
loss: 1.292222  [51264/60000]
loss: 1.189494  [57664/60000]
Test Error: 
 Accuracy: 64.0%, Avg loss: 1.222897 

Epoch 3
-------------------------------
loss: 1.296676  [   64/60000]
loss: 1.285180  [ 6464/60000]
loss: 1.113040  [12864/60000]
loss: 1.233147  [19264/60000]
loss: 1.103150  [25664/60000]
loss: 1.129982  [32064/600